In [ ]:
# 环境安装 (需先执行)
# !pip install transformers datasets peft accelerate torch

from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, TaskType, get_peft_model
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score

# 步骤1: 加载模型和tokenizer
model_name = "distilbert-base-uncased"  # 使用较小的模型示例
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 步骤2: 配置LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # 任务类型
    r=8,                         # LoRA秩
    lora_alpha=32,               # 缩放因子
    lora_dropout=0.1,            # Dropout概率
    bias="none",                 # 是否训练偏置项
    target_modules=["q_lin", "v_lin"]  # 对Transformer的query和value层应用LoRA
)

# 将基础模型转换为PEFT模型（仅添加0.7%的可训练参数！）
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()  # 输出: trainable params: 884,736 || all params: 66,951,936 || 1.32%

# 步骤3: 准备数据集
dataset = load_dataset("imdb")
def tokenize_fn(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize_fn, batched=True)
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))  # 小样本示例
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(200))

# 步骤4: 定义评估指标
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# 步骤5: 配置训练参数
training_args = TrainingArguments(
    output_dir="./lora_imdb",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    save_strategy="no",  # 小样本训练不需要保存检查点
    fp16=True,           # 启用混合精度训练
)

# 步骤6: 创建Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# 步骤7: 训练模型
trainer.train()

# 步骤8: 测试推理示例
sample_text = "This movie was absolutely fantastic! The acting was superb."
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    logits = peft_model(**inputs).logits
prediction = torch.argmax(logits, dim=1).item()
print(f"Prediction: {'Positive' if predction == 1 else 'Negative'}")

In [ ]:
# 对每个任务创建独立的LoRA配置
task1_lora = LoraConfig(...)
task2_lora = LoraConfig(...)

# 训练时动态切换适配器
peft_model.set_adapter("task1")  # 切换至任务1的适配器
peft_model.set_adapter("task2")  # 切换至任务2的适配器

In [ ]:
# 常规全参数微调的参数量：
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
sum(p.numel() for p in model.parameters() if p.requires_grad)  # 约6700万可训练参数

In [ ]:
merged_model = peft_model.merge_and_unload()  # 将LoRA权重合并到基础模型
merged_model.save_pretrained("./merged_model")